In [1]:
# %pip install pandas
# %pip install numpy
# %pip install glob
# %pip install matplotlib

import pandas as pd
import numpy as np
import glob
# import matplotlib.pyplot as plt

In [2]:
game_state_totals = pd.read_csv('./res/game-state-totals.csv').set_index('Unnamed: 0').rename_axis('game_state')
expected_game_state_delta_wpa = pd.read_csv('./res/expected-game-state-delta-wpa.csv').set_index('Unnamed: 0').rename_axis('game_state')

print(game_state_totals.head())
print(expected_game_state_delta_wpa.head())

               N  Wins        WP      freq
game_state                                
1_0_0_0_-10   22     1  0.045455  0.000006
1_1_0_0_-10    0     0       NaN  0.000000
2_0_0_0_-10   94     2  0.021277  0.000027
2_1_0_0_-10   22     0  0.000000  0.000006
3_0_0_0_-10  262     5  0.019084  0.000076
             expected_delta_wpa        LI      freq
game_state                                         
1_0_0_0_-10            0.045455  0.952895  0.000006
2_0_0_0_-10            0.012002  0.251603  0.000027
2_1_0_0_-10            0.038871  0.814880  0.000006
3_0_0_0_-10            0.015140  0.317390  0.000076
3_1_0_0_-10            0.007905  0.165728  0.000024


In [3]:
def create_game_states(innings, tops, outs, base_codes, score_diffs):
    game_states = []
    for i in range(len(innings)):
        game_states.append(f'{innings[i]}_{tops[i]}_{outs[i]}_{base_codes[i]}_{score_diffs[i]}')
    return game_states

csv_files = glob.glob('../R/data/raw-pbps/*/*.csv', recursive=True)

li = []
pbps = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)
pbps['game_state'] = create_game_states(
        pbps['inning'], 
        pbps['top_inning'], 
        pbps['outs_before'], 
        pbps['base_cd_before'], 
        np.maximum(np.minimum(pbps['home_score_before'] - pbps['away_score_before'], 10), -10)
        )

pbps['next_game_state'] = pbps['game_state'].shift(-1)

# print(pbps.shape)
pbps = pbps[pbps['game_state'].notnull()]
pbps = pbps[pbps['next_game_state'].notnull()]
pbps = pbps[pbps['bat_name'].notnull()]
pbps = pbps[pbps['pitcher_name'].notnull()]
print(pbps.shape)

pbps['WPA'] = pbps.apply(lambda row: game_state_totals.loc[row['next_game_state'], 'WP'] - game_state_totals.loc[row['game_state'], 'WP'], axis=1)
pbps['LI'] = pbps.apply(lambda row: expected_game_state_delta_wpa.loc[row['game_state'], 'LI'], axis=1)
pbps['WPA / LI'] = pbps['WPA'] / pbps['LI']

pbps.to_csv('./res/pbps.csv')
pbps.head()

/var/folders/jy/rwh1dlwn46j92r_cl360mtkw0000gn/T/ipykernel_44876/1098301961.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pbps = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)


(736603, 28)


,year,home_team,home_team_id,away_team,away_team_id,inning,inning_top_bot,top_inning,event_cd,bat_name,...,base_cd_before,base_cd_after,game_end,new_game,won,game_state,next_game_state,WPA,LI,WPA / LI
0,2020,Tulane,718,Middle Tenn.,419,1,top,1,3,Escher,...,0,0,0,1,1,1_1_0_0_0,1_1_1_0_0,0.027296,0.816262,0.033441
1,2020,Tulane,718,Middle Tenn.,419,1,top,1,2,Thomas,...,0,0,0,0,1,1_1_1_0_0,1_1_2_0_0,0.027260,0.793459,0.034355
2,2020,Tulane,718,Middle Tenn.,419,1,top,1,2,Engel,...,0,0,0,0,1,1_1_2_0_0,1_0_0_0_0,-0.010059,0.655260,-0.015351
3,2020,Tulane,718,Middle Tenn.,419,1,bot,0,2,HASKIN,...,0,0,0,0,1,1_0_0_0_0,1_0_1_0_0,-0.020707,0.883569,-0.023436
4,2020,Tulane,718,Middle Tenn.,419,1,bot,0,2,G. MATHEWS,...,0,0,0,0,1,1_0_1_0_0,1_0_2_0_0,-0.026691,0.880382,-0.030317


In [ ]:
pd.set_option('display.max_columns', None)
pbps.head()